# Decision Trees - Edward King

In this section I will explore the Decision Tree Model of machine learning.

### Contents:

* Graph theory
* Decision trees
* Information theory
* Practicalities
* The Code

Chapter 1 - Graph theory
---

### 1.1 - Graph theory basics ###

Graph theory is the field of mathematics dedicated to studying the structure and properties of graphs. In order to understand Decision Trees, we first need to know what a tree is. In this section, I will provide the definitions needed to rigourously define a tree.

>#### Definition 1.1: Graphs, Nodes, Arcs and Direction
>A __graph__ is an ordered pair of sets $(V,\ E)$ where:
>* $V$ is a set of __nodes__ (or vertices)
>* $E$ is a set of __arcs__ (or edges) which are ordered pairs of vertices, i.e. $E=\{(x,\ y):x,y\in V\}$.
>
>It is important to note that arcs are directional: $(x,\ y)\neq(y,\ x)$.  
>We say a graph is __directed__ if $\exist (x,\ y)\in E\ s.t.\ (y,\ x)\notin E$.  
>A graph is __undirected__ if $\forall (x,\ y)\in E,\ \exist (y,\ x)\in E$.

![alt text](images/LetteredGraph.png)  
_Figure 1.2 - an undirected graph._

By convention, if a graph is directed it is shown using arrows on the arcs. Therefore, we may assume that this graph is undirected.

Next we need a notion of cycles in a graph. To construct this, we need a series of definitions.

>#### Definition 1.3: Walks, Trails, Paths, Closure, Cycles and Connectedness
>* A __walk__ is an ordered collection of nodes, $(x_1,\ x_2,\ x_3,\ldots,\ x_n)$.  
>Every walk has a corresponding unique collection of arcs, $(e_1,\ e_2,\ldots,\ e_{n-1})$ where $e_i = (x_i,\ x_{i+1})$.  
>N.B. we only consider walks of finite length.
>
>* A __trail__ is a walk where the collection of arcs has no repeats i.e. $i\neq j\Rightarrow e_i\neq e_j,\ \forall i,j: 0≤i,j≤n-1$.
>
>* A __path__ is a trail where the collection of nodes has no repeats i.e. $i\neq j\Rightarrow x_i\neq x_j,\ \forall i,j: 0≤i,j≤n-1$.
>
>* A walk is __closed__ if $x_1=x_n$.
>
>* A __cycle__ is a closed path.
>
>* A graph is __cyclic__ if it has a cycle.
>
>* A graph is __acyclic__ if it has no cycles.
>
>* Two nodes, $x_a,\ x_b$, are __connected__ if there is a walk of the form $(x_a,\ x_i,\ x_{i+1}, \ldots,\ x_b)$.
>
>* A graph is __connected__ if each pair of nodes ($x,\ y\in V$) is connected.

Visually we can see that there are multiple cycles in the graph in figure 1.2, e.g. $(A,\ B,\ C,\ E,\ A)$ and $(A,\ D,\ E,\ A)$, meaning that this graph is cyclic.

>#### Definition 1.4: Trees, Roots, Parents, Children, Leaves and Binary Trees
>* A __tree__ is a connected, acyclic graph.
>
>* In a tree, we designate one node to be the __root__.
>
>* Given two nodes $x,\ y\in V$, if there is a path from $x$ to the root which passes through $y$ then we say that $y$ is the __parent node__ of $x$, the __child node__.
>
>* If a node has no children then it is a __leaf node__.
>
>* If each node in a tree has no more than two children then it is a __binary tree__.

![alt text](images/trees.png)  
_Figure 1.5 - three trees (the one on the right is a binary tree)._

N.B. Only nodes with at most two arcs can be designated as the root of a binary tree however any node can be designated the root of a non-binary tree.

At last we have all the pieces needed to start on decision trees.

Chapter 2 - Decision Trees
---

### 2.1 - Decision tree basics
A decision tree acts on a set of data where each data point is given by an ordered set $(a_1,\ a_2,\ \ldots ,\ a_n,\ c)$ where each $a_i$ is a feature and $c$ is the classification of this data point. One of the main advantages of using a decision tree is that there is no pre-processing of data as each feature can be of any data type (i.e. continuous, discrete), though for simplicity I will assume that the data features are all continuous in my code.

A decision tree is a tree (or tree-like structure) where each node in the set $V$ describes a condition $C$ on a data feature (for example: $a_1\leq 1$, or $a_2 =$ "Honda") which indicates how data should be split as you traverse the tree (with the given examples: feature $a_1$ is less than or equal to $1$ or $a_2$ is "Honda"). If the node is a leaf node then the condition states what classification should be assigned to the data point.

![alt text](images/DecisionTree.png)  
_Figure 2.1 - A simple decision tree_

Consider the well known model, the Galton board, shown in figure 2.2 below. Consider each ball to be a data point which traverses the decision tree by falling under gravity. The pocket it falls into at the bottom determines which class it is assigned to. Each peg in the board represents a node in the decision tree that decides which path the data point should take down the board.

![alt text](images/GaltonBoard.png)  
_Figure 2.2 - Galton board_

### 2.2 - Decision trees in the wild
This definition allows for a wide range of use cases beyond the one detailed later. This concept is used frequently in game development to design the behaviour of non-player characters and enemies. This is clear in the board game Gloomhaven where players are given a decision tree to determine how the enemies move and attack the players. Each enemy has its own set of information (such as whether or not it is stunned, its position on the board, etc.) which can be considered as the features $(a_1,\ a_2,\ \ldots ,\ a_n)$ and the classification, $c$, is the action the enemy will take. The image below could be formalised to look more like figure 2.1 but I shall leave this as an exercise to the reader.

![alt text](images/gloomhaven_behaviour_tree.png)  
_Figure 2.3 - Gloomhaven behaviour tree_

### 2.3 - Back to machine learning ###
Now we return to the meat of this section: how decision trees are applied to machine learning. I will explore the "learning" as it applies here in the next section but first I will discuss some reasons why one might use decision trees over other models.

Advantages:
* Unlike other models (particularly neural networks), it is very easy to see how a decision tree comes to its conclusions.
* As mentioned in §2.1, the data features can be either discrete or continuous.
* Can be used in both classification and regression problems (though I'm only considering classification problems here).
* Fast query time - on average this has complexity $O(\log{n_{samples}})$[1].

Disadvantages:
* Use of a greedy approach (the best choice is chosen at each stage without considering the effect on future choices). This can lead to a non-optimal solution but will be faster than a more comprehensive search.
* Small variations in the input data can lead to wildly different trees.
* Slow construction time - on average this has complexity $O(n_{samples}n_{features}\log(n_{samples}))$[1]. This issue can be mitigated since we only construct the tree once then query it from then on.
* Cannot add data to the tree once constructed - unlike KNN, our model cannot be improved once constructed.
* Prone to overfitting - (see Tobey's section for a more detailed description of the issue). Some remedies to this are discussed at the end of §2.4.


### 2.4 - Algorithm for constructing a decision tree ###
Now that we know what a decision tree looks like, can we construct one? Below I detail the algorithm for constructing a decision tree, italicised lines will be expanded upon later:
>1. Receive a dataset. _Select stopping requirements_.
>
>2. Create a root node, assign the whole dataset to it and mark it as incomplete and active.
>
>3. _Choose a condition on which to split the data assigned to the active node_.
>
>4. Assign the chosen condition to the active node.
>
>5. Create two child nodes of the active node and mark each as incomplete and inactive.
>
>6. Assign the data that satisfies the condition to the first child and the remaining data to the second.
>
>7. Mark the active node as complete and inactive.
>
>8. If the stopping requirements have been met proceed to step 11, otherwise proceed to step 9.
>
>9. Select any incomplete node and set its status to active.
>
>10. Proceed to step 3.
>
>11. Select any incomplete node, mark it as complete and set its value to the mode class in its subset of the dataset. It is now a 
>leaf node.
>
>12. If all nodes are complete proceed to step 13, otherwise proceed to step 11.
>
>13. Terminate the algorithm.

This algorithm can be optimised using recursion (which I will use in my code), though for clarity I use this slightly more inefficient version here.

As promised, I will expand on step 1 now. Step 3 will be left for its own section.

Step 1 is used as a method of preventing overfitting as well as enabling termination of the algorithm. There are two requirements that have to be met in order to terminate the algorithm:
1. Maximum distance from the root - this prevents the algorithm continuing until all data has been classified perfectly.
2. Minimum sample size - if the data subset assigned to a node has too few elements then we stop the algorithm to prevent overfitting.

I will now explain step 3 of the algorithm.

Chapter 3 - Choosing Optimal Splitting Conditions
---

### Information theory and entropy
Given two choices of splitting conditions, how do we evaluate which is more effective at splitting the data. For this problem, 


\begin{align}
E(x) = \sum_{c\in C} -(p_c) log_2(p_c)
\end{align}
Where $p_c$ is the probability of class $c$.\
$IG = Entropy(Parent) - \sum w_iEntropy(Child)$

Part 4 - The Code
---

In [27]:
import numpy as np
import pandas as pd

Old code
---

In [20]:
import numpy as np
import pandas as pd

In [21]:
# col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
# data = pd.read_csv('iris.data', sep=',', header=None, names=col_names)
col_names = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4', 'Class']
data = pd.read_csv('Dry_Bean_Dataset.data', sep=',', header=None, names=col_names)

data.head(10)

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
5,30279,634.927,212.560556,181.510182,1.171067,0.520401,30600,196.347702,0.775688,0.989510,0.943852,0.923726,0.007020,0.003153,0.853270,0.999236,SEKER
6,30477,670.033,211.050155,184.039050,1.146768,0.489478,30970,196.988633,0.762402,0.984081,0.853080,0.933374,0.006925,0.003242,0.871186,0.999049,SEKER
7,30519,629.727,212.996755,182.737204,1.165591,0.513760,30847,197.124320,0.770682,0.989367,0.967109,0.925480,0.006979,0.003158,0.856514,0.998345,SEKER
8,30685,635.681,213.534145,183.157146,1.165852,0.514081,31044,197.659696,0.771561,0.988436,0.954240,0.925658,0.006959,0.003152,0.856844,0.998953,SEKER
9,30834,631.934,217.227813,180.897469,1.200834,0.553642,31120,198.139012,0.783683,0.990810,0.970278,0.912125,0.007045,0.003008,0.831973,0.999061,SEKER


In [22]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        
        # For decision nodes
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # For leaf nodes
        self.value = value


In [23]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        
        # initialise the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def BuildTree(self, dataset, curr_depth=0): 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:

            # find the best split
            best_split = self.GetBestSplit(dataset, num_samples, num_features)

            # check if information gain is positive
            if best_split["info_gain"]>0:

                # recur left
                left_subtree = self.BuildTree(best_split["dataset_left"], curr_depth+1)

                # recur right
                right_subtree = self.BuildTree(best_split["dataset_right"], curr_depth+1)

                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.CalculateLeafValue(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def GetBestSplit(self, dataset, num_samples, num_features):
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.Split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.InformationGain(y, left_y, right_y)
                    # update the best split if necessary
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def Split(self, dataset, feature_index, threshold):
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def InformationGain(self, parent, l_child, r_child, mode="entropy"):
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.GiniIndex(parent) - (weight_l*self.GiniIndex(l_child) + weight_r*self.GiniIndex(r_child))
        else:
            gain = self.Entropy(parent) - (weight_l*self.Entropy(l_child) + weight_r*self.Entropy(r_child))
        return gain
    
    def Entropy(self, y):
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def GiniIndex(self, y):
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def CalculateLeafValue(self, Y):

        Y = list(Y)
        return max(Y, key=Y.count)
    
    def PrintTree(self, tree=None, indent=" "):
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)

            print("%sleft:" % (indent), end="")
            self.PrintTree(tree.left, indent + indent)
            
            print("%sright:" % (indent), end="")
            self.PrintTree(tree.right, indent + indent)
    
    def Fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.BuildTree(dataset)
    
    def Predict(self, X):
        preditions = [self.MakePrediction(x, self.root) for x in X]
        return preditions
    
    def MakePrediction(self, x, tree):

        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.MakePrediction(x, tree.left)
        else:
            return self.MakePrediction(x, tree.right)

In [24]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=1000, test_size=10, random_state=41)

In [25]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=4)
classifier.Fit(X_train,Y_train)
classifier.PrintTree()

X_2 <= 337.53295866964567 ? 0.7862887945200716
 left:X_3 <= 181.27760357155148 ? 0.6030206435481191
  left:X_2 <= 274.49470044845833 ? 0.2503655606801769
    left:X_2 <= 256.0466415892121 ? 0.07010591896916663
        left:X_12 <= 0.0073734377548812 ? 0.05971828222415425
                left:DERMASON
                right:DERMASON
        right:X_8 <= 0.7161057692307692 ? 0.07979388599982407
                left:DERMASON
                right:DERMASON
    right:X_4 <= 1.7643227793589036 ? 0.47392066312979675
        left:X_15 <= 0.9937465421501556 ? 0.2602739159159402
                left:SIRA
                right:DERMASON
        right:X_9 <= 0.988102632710593 ? 0.5435644431995964
                left:HOROZ
                right:SIRA
  right:X_13 <= 0.0021023489782095 ? 0.7381502273188227
    left:X_3 <= 210.26952899304652 ? 0.2182950950516962
        left:X_0 <= 42147.0 ? 0.16854484493241406
                left:SIRA
                right:SIRA
        right:X_10 <= 0.851706144325493

In [26]:
Y_pred = classifier.Predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8

Part 5 - Potential Improvements
---

__Add something about discrete data features and random forest, xgboost, etc.__

Glossary
===

Acyclic graph: A graph that contains no cycles.

Arc: A line connecting two nodes on a graph.

Binary tree: A tree where each node has at most two child nodes.

Child node: A node with a parent.

Connected nodes: Two nodes with an arc between them.

Connected graph: A graph where there is a path between any pair of nodes.

Cycle: A path that begins and ends at the same node.

Cyclic graph: A graph that contains a cycle.

Directed arc: An arc which can only be traversed in a single direction.

Directed graph: A graph where at least one arc is directed.

Graph: An ordered pair $(V, E)$ where $V$ is a set of vertices and $E$ a set of ordered pairs of vertices indicating connections between vertices.

Leaf node: A node which has no child nodes.

Node: A generic .

Parent node: The first node reached on the path from a node to the root. Nodes can have at most one parent. The root node has no parents.

Path: A sequence of nodes where all pairs of consecutive nodes are connected and no arc or node is repeated.

Root node: A node that has been designated as the root.

Tree: An acyclic, connected, undirected graph.

Undirected arc: An arc which can be traversed in either direction.

Undirected graph: A graph where all arcs are undirected.

References:
---

[1]: https://scikit-learn.org/stable/modules/tree.html